In [11]:
import os
import pandas as pd
import numpy as np

import cv2


In [12]:
feat_path = './ML_Features_1144.csv'
df = pd.read_csv(feat_path)

In [13]:
df.shape

(1144, 69)

In [14]:
df.head()

,Unnamed: 0,image.name,X.x,Blue.count,red.count,Blue.percentage,red.percentage,total.clusters,average.clusters.32,area,...,Texture_SumEntropy_3_90,Texture_SumVariance_3_0,Texture_SumVariance_3_135,Texture_SumVariance_3_45,Texture_SumVariance_3_90,Texture_Variance_3_0,Texture_Variance_3_135,Texture_Variance_3_45,Texture_Variance_3_90,classification
0,1,Case 3 A10-10547-25283,548,16611,52475,1.584148,5.004406,123,0.112844,81.252033,...,1.269986,4.985359,4.654160,4.499266,4.718412,1.545288,1.538943,1.542346,1.541256,Non-Tumor
1,2,Case 3 A10-10566-40206,549,93148,282307,8.883286,26.922894,143,0.131193,69.671329,...,1.689416,24.047674,23.805470,23.886660,23.949666,6.115417,6.105633,6.105821,6.109401,Non-Tumor
2,3,Case 3 A10-13444-20223,550,107853,198888,10.285664,18.967438,166,0.152294,75.361446,...,1.668477,23.754724,23.636317,23.338890,23.673391,6.170840,6.175347,6.176097,6.171856,Non-Tumor
3,4,Case 3 A10-14507-37285,551,58609,208594,5.589390,19.893074,153,0.140367,68.398693,...,1.839611,20.562092,20.061859,19.908849,20.037982,5.341032,5.336172,5.342228,5.338274,Non-Tumor
4,5,Case 3 A10-14726-26052,552,13419,56428,1.279736,5.381393,81,0.074312,82.679012,...,1.197788,3.977051,3.552465,3.481328,3.593105,1.252694,1.253966,1.253085,1.253502,Non-Tumor


In [15]:
print(df[df['classification']=='Viable'].shape)
print(df[df['classification']=='Non-Viable-Tumor'].shape)
print(df[df['classification']=='Non-Tumor'].shape)

(292, 69)
(263, 69)
(536, 69)


In [42]:
new_df = df.copy()
all_classes = df['classification'].tolist()
print(set(all_classes))

cnt = 0
corrected_classes = list()
for c in all_classes:
    if c=='viable: non-viable':
        cnt += 1
        corrected_classes.append('Viable')
    else:
        corrected_classes.append(c)
        
print(set(corrected_classes))
new_df['classification'] = corrected_classes
        

{'Non-Tumor', 'Viable', 'Non-Viable-Tumor', 'viable: non-viable'}
{'Non-Tumor', 'Viable', 'Non-Viable-Tumor'}


In [43]:
print(new_df[new_df['classification']=='Viable'].shape)
print(new_df[new_df['classification']=='Non-Viable-Tumor'].shape)
print(new_df[new_df['classification']=='Non-Tumor'].shape)

(345, 69)
(263, 69)
(536, 69)


In [44]:
new_df.head()

,Unnamed: 0,image.name,X.x,Blue.count,red.count,Blue.percentage,red.percentage,total.clusters,average.clusters.32,area,...,Texture_SumEntropy_3_90,Texture_SumVariance_3_0,Texture_SumVariance_3_135,Texture_SumVariance_3_45,Texture_SumVariance_3_90,Texture_Variance_3_0,Texture_Variance_3_135,Texture_Variance_3_45,Texture_Variance_3_90,classification
0,1,Case 3 A10-10547-25283,548,16611,52475,1.584148,5.004406,123,0.112844,81.252033,...,1.269986,4.985359,4.654160,4.499266,4.718412,1.545288,1.538943,1.542346,1.541256,Non-Tumor
1,2,Case 3 A10-10566-40206,549,93148,282307,8.883286,26.922894,143,0.131193,69.671329,...,1.689416,24.047674,23.805470,23.886660,23.949666,6.115417,6.105633,6.105821,6.109401,Non-Tumor
2,3,Case 3 A10-13444-20223,550,107853,198888,10.285664,18.967438,166,0.152294,75.361446,...,1.668477,23.754724,23.636317,23.338890,23.673391,6.170840,6.175347,6.176097,6.171856,Non-Tumor
3,4,Case 3 A10-14507-37285,551,58609,208594,5.589390,19.893074,153,0.140367,68.398693,...,1.839611,20.562092,20.061859,19.908849,20.037982,5.341032,5.336172,5.342228,5.338274,Non-Tumor
4,5,Case 3 A10-14726-26052,552,13419,56428,1.279736,5.381393,81,0.074312,82.679012,...,1.197788,3.977051,3.552465,3.481328,3.593105,1.252694,1.253966,1.253085,1.253502,Non-Tumor


In [45]:
set_classes = list(set(corrected_classes))
new_df = new_df[['image.name', 'classification']]
new_df.columns = ['name', 'class']
new_df['label'] = [
    set_classes.index(c) for c in new_df['class']
]
new_df['name'] = [
    '-'.join(filter(lambda x: (x != '' and x != ' '), n.replace(' ', '-').split('-'))) for n in new_df['name']
]
new_df.head()

,name,class,label
0,Case-3-A10-10547-25283,Non-Tumor,0
1,Case-3-A10-10566-40206,Non-Tumor,0
2,Case-3-A10-13444-20223,Non-Tumor,0
3,Case-3-A10-14507-37285,Non-Tumor,0
4,Case-3-A10-14726-26052,Non-Tumor,0


In [46]:
for n in new_df['name']:
    if 'Case-4' in n or 'Case 4' in n:
        print(n)

Case-4-C21-11229-17093
Case-4-C21-13339-24727
Case-4-C21-14246-18799
Case-4-C21-17996-22366
Case-4-C21-20675-9095
Case-4-C21-22035-24818
Case-4-C21-22607-7946
Case-4-C21-23743-36254
Case-4-C21-25306-21689
Case-4-C21-25396-25798
Case-4-C21-26326-8633
Case-4-C21-27726-37043
Case-4-C21-28314-22979
Case-4-C21-29101-18901
Case-4-C21-29849-33988
Case-4-C21-30599-31986
Case-4-C21-34438-19627
Case-4-C21-34899-31136
Case-4-C21-35538-25356
Case-4-C21-37411-12734
Case-4-C21-38458-18002
Case-4-C21-39931-19569
Case-4-C21-40652-23162
Case-4-C21-42043-16377
Case-4-C21-43834-23094
Case-4-C21-44291-25760
Case-4-C21-45309-20604
Case-4-C22-12113-23584
Case-4-C22-13293-17406
Case-4-C22-14193-24204
Case-4-C22-14852-25038
Case-4-C22-18690-17829
Case-4-C22-19399-25392
Case-4-C22-19841-14408
Case-4-C22-20288-31102
Case-4-C22-20420-19004
Case-4-C22-21902-8062
Case-4-C22-22350-18241
Case-4-C22-25017-13207
Case-4-C22-25541-21407
Case-4-C22-25642-6919
Case-4-C22-26476-28003
Case-4-C22-29668-36352
Case-4-C22-29696

In [47]:
print(set_classes)

['Non-Tumor', 'Viable', 'Non-Viable-Tumor']


In [48]:
new_df.to_csv('./label_mapping.csv', index=False)

In [49]:
label_df = pd.read_csv('./label_mapping.csv')
label_df.head()

,name,class,label
0,Case-3-A10-10547-25283,Non-Tumor,0
1,Case-3-A10-10566-40206,Non-Tumor,0
2,Case-3-A10-13444-20223,Non-Tumor,0
3,Case-3-A10-14507-37285,Non-Tumor,0
4,Case-3-A10-14726-26052,Non-Tumor,0


In [52]:
def stratified_sampling(df, train_rate=0.6, valid_rate=0.2, num_classes=3):
    class_indices = [
        df[df['label']==c].index.tolist() for c in range(num_classes)
    ]
    test_rate = 1 - train_rate - valid_rate
    train_indices = []
    test_indices = []
    valid_indices = []
    for label in range(num_classes):
        current_label_indices = class_indices[label]
        np.random.shuffle(current_label_indices)
        threshold1 = int(len(current_label_indices) * train_rate)
        threshold2 = int(len(current_label_indices) * (train_rate + valid_rate))
        train_idx = current_label_indices[:threshold1]
        valid_idx = current_label_indices[threshold1: threshold2]
        test_idx = current_label_indices[threshold2:]
        
        train_indices += train_idx
        valid_indices += valid_idx
        test_indices += test_idx
        
    train_df = df.iloc[train_indices].reset_index(drop=True)
    test_df = df.iloc[test_indices].reset_index(drop=True)
    valid_df = df.iloc[valid_indices].reset_index(drop=True)
    
    return train_df, valid_df, test_df
    

In [53]:
train_df, valid_df, test_df = stratified_sampling(label_df)
train_df.head()

,name,class,label
0,P9-B11-26343-18701,Non-Tumor,0
1,P9-B6-5195-29864,Non-Tumor,0
2,Case-3-A15-10555-31474,Non-Tumor,0
3,P9-B28-6717-20778,Non-Tumor,0
4,P9-B7-23277-6246,Non-Tumor,0


In [55]:
train_df.to_csv('./train_mapping.csv', index=False)
valid_df.to_csv('./valid_mapping.csv', index=False)
test_df.to_csv('./test_mapping.csv', index=False)

In [56]:
df_ = pd.read_csv('./train_mapping.csv')
df_.head()

,name,class,label
0,P9-B11-26343-18701,Non-Tumor,0
1,P9-B6-5195-29864,Non-Tumor,0
2,Case-3-A15-10555-31474,Non-Tumor,0
3,P9-B28-6717-20778,Non-Tumor,0
4,P9-B7-23277-6246,Non-Tumor,0


In [57]:
print(train_df[train_df['class']=='Viable'].shape)
print(train_df[train_df['class']=='Non-Viable-Tumor'].shape)
print(train_df[train_df['class']=='Non-Tumor'].shape)

(207, 3)
(157, 3)
(321, 3)


### Process data